In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

# Setting the pyecharts config
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_LAB

# setting pyecharts' charts
from pyecharts.charts import Funnel, Bar, Line, Grid, Pie
from pyecharts import options as opts
from pyecharts.globals import ThemeType

# Jupyter Magic Func
%matplotlib inline

In [3]:
#importing data
df = pd.read_csv('Data/UserBehavior.csv', header=None, parse_dates=True)

# rename columns
df.columns = ['User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type', 'Time']

In [4]:
df['Time'] = pd.to_datetime(df['Time'], unit='s')

In [ ]:
# trans the dtype
df['Time'] = pd.to_datetime(df['Time'], unit='s')
df['Time'] = df.Time.astype('str')

#split the columns
df['Time_Split'] = df.Time.str.split(' ')
df['Date'] = df.Time_Split.str.get(0)
df['Hour'] = df.Time_Split.str.get(1)

# Drop the Column
df = df.drop(['Time_Split'], axis=1)

# Redefine the order of columns 
df = df[['Date', 'Hour', 'Time', 'User_Id', 'Item_Id', 'Category_Id', 'Behavior_Type']]

# Tranform the dtype back to datetime, in order to split the "Date" Column to two columns.
df['Date'] = pd.to_datetime(df['Date'])
df['Time'] = pd.to_datetime(df['Time'])

# Clean the wrong data range. Ex. outer the range from 2017,11,23 to 2017,12,24.
mask_1 = df['Date'] >= pd.to_datetime('2017-11-25')
mask_2 = df['Date'] <= pd.to_datetime('2017-12-02')
df = df[mask_1 & mask_2]

## 3. RFM 模型分析
透過 RFM 模型找出對營收貢獻最大的用戶，能夠透過該用戶數據，進行用戶畫像的描繪。在本數據中，因商品與營收等資料都已匿名化，因此主要分析 R（Recency）與 F（Frequency）進行分析。

### i. 最近購買時間
Rencency 會根據
會將近一個月的顧客分成五等分，每一等分為資料庫的 20%。
該分析便是找出至今再次購買與上次購買產品的時間差的 20% 用戶。換句話說，距上次消費時間越近的用戶，會被分到更高的等級。

從數距上可以發現最多的共有

In [5]:
df_rfm = df[df['Behavior_Type'] == 'buy'][['User_Id', 'Time']]
df_rfm['Date'] = df_rfm.Time.dt.strftime('%Y-%m-%d')

In [12]:
r = df_rfm.groupby('User_Id')['Time'].max().reset_index()
r

,User_Id,Time
0,2,2017-12-02 11:35:42
1,4,2017-11-30 02:23:20
2,11,2017-11-25 13:16:32
3,16,2017-12-01 15:56:21
4,17,2017-12-01 05:28:43
...,...,...
672399,1017999,2017-12-02 09:45:11
672400,1018002,2017-12-03 07:16:24
672401,1018009,2017-12-01 04:18:02
672402,1018010,2017-12-02 09:34:57


In [13]:
r['R'] = (pd.to_datetime('2017-12-04') - r['Time']).dt.days
r = r[['User_Id', 'R']]

In [14]:
r.head()

,User_Id,R
0,2,1
1,4,3
2,11,8
3,16,2
4,17,2


In [15]:
r.R.value_counts()

0    160606
1    134398
2     84990
3     73566
4     61007
5     48322
6     42377
7     36209
8     28894
9      2035
Name: R, dtype: int64

In [16]:
f = df_rfm[['User_Id', 'Date', 'Time']]
f = f.groupby(['User_Id', 'Date'])['Time'].count().reset_index()
f = f.groupby('User_Id')['Date'].count().reset_index()
f

,User_Id,Date
0,2,3
1,4,2
2,11,1
3,16,2
4,17,1
...,...,...
672399,1017999,3
672400,1018002,1
672401,1018009,2
672402,1018010,4


In [58]:
# f_dup = f.groupby(['User_Id', 'Date'])['Time'].count().reset_index()
# f = f_dup.groupby('User_Id')['Date'].count().reset_index()

# f = f_dup.groupby('User_Id')['Hour'].count().reset_index()

In [17]:
f.columns = ['Id', 'F']
f

,Id,F
0,2,3
1,4,2
2,11,1
3,16,2
4,17,1
...,...,...
672399,1017999,3
672400,1018002,1
672401,1018009,2
672402,1018010,4


In [18]:
rfm = pd.merge(r, f, left_on="User_Id", right_on="Id", how="inner")

In [19]:
rfm.drop("Id", axis=1, inplace=True)
rfm

,User_Id,R,F
0,2,1,3
1,4,3,2
2,11,8,1
3,16,2,2
4,17,2,1
...,...,...,...
672399,1017999,1,3
672400,1018002,0,1
672401,1018009,2,2
672402,1018010,1,4


In [20]:
rfm = rfm[['User_Id', 'R', 'F']]
rfm.head()

,User_Id,R,F
0,2,1,3
1,4,3,2
2,11,8,1
3,16,2,2
4,17,2,1


In [21]:
rfm['R_Score'] = pd.cut(rfm['R'], bins=[0, 3, 6, 11], labels=[3,2,1], right=False).astype('float')

In [22]:
rfm['F_Score'] = pd.cut(rfm['F'], bins=[1, 2, 3, 4, 5, 100000], labels=[5,4,3,2,1], right=False).astype('float')

In [23]:
rfm

,User_Id,R,F,R_Score,F_Score
0,2,1,3,3.0,3.0
1,4,3,2,2.0,4.0
2,11,8,1,1.0,5.0
3,16,2,2,3.0,4.0
4,17,2,1,3.0,5.0
...,...,...,...,...,...
672399,1017999,1,3,3.0,3.0
672400,1018002,0,1,3.0,5.0
672401,1018009,2,2,3.0,4.0
672402,1018010,1,4,3.0,2.0


In [24]:
rfm['R_Big_then_Avg'] = (rfm.R_Score > rfm.R_Score.mean()) * 1
rfm['F_Big_then_Avg'] = (rfm.F_Score > rfm.F_Score.mean()) * 1
rfm['Type'] = (rfm['R_Big_then_Avg'] * 10 + rfm['F_Big_then_Avg'] * 1)

In [25]:
rfm['R'] = rfm.R.astype('int8')
rfm['F'] = rfm.F.astype('int8')
rfm['R_Big_then_Avg'] = rfm['R_Big_then_Avg'].astype('float16')
rfm['F_Big_then_Avg'] = rfm['F_Big_then_Avg'].astype('float16')

rfm.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 672404 entries, 0 to 672403
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   User_Id         672404 non-null  int64  
 1   R               672404 non-null  int8   
 2   F               672404 non-null  int8   
 3   R_Score         672404 non-null  float64
 4   F_Score         672404 non-null  float64
 5   R_Big_then_Avg  672404 non-null  float16
 6   F_Big_then_Avg  672404 non-null  float16
 7   Type            672404 non-null  int64  
dtypes: float16(2), float64(2), int64(2), int8(2)
memory usage: 29.5 MB


In [26]:
rfm.head()

,User_Id,R,F,R_Score,F_Score,R_Big_then_Avg,F_Big_then_Avg,Type
0,2,1,3,3.0,3.0,1.0,0.0,10
1,4,3,2,2.0,4.0,0.0,0.0,0
2,11,8,1,1.0,5.0,0.0,1.0,1
3,16,2,2,3.0,4.0,1.0,0.0,10
4,17,2,1,3.0,5.0,1.0,1.0,11


In [31]:
mapping_type = {
    0: "流失用戶",
    1: "喚回用戶",
    10: "潛在消費用戶",
    11: "重要價值客戶"
}

user_type = rfm.Type.value_counts().reset_index().rename({'index': "Type", "Type": "Count"}, axis=1)
user_type['Type'] = user_type.Type.map(mapping_type)

In [32]:
user_type

,Type,Count
0,潛在消費用戶,262799
1,喚回用戶,184509
2,重要價值客戶,117195
3,流失用戶,107901


In [33]:
user_type['Ratio'] = round(100 * user_type.Count / user_type.Count.sum(), 1)

In [34]:
def rfm_chart():
    pie = (
        Pie()
        .add("", [list(z) for z in zip(user_type.Type.to_list(), user_type.Ratio.to_list())])
        .set_global_opts(title_opts=opts.TitleOpts(title="RF 用戶分類圖"))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}%"))
    )
    return pie
rfm_chart().render_notebook()

* 流失用戶：是指近期低於消費標準，且也低於平均的消費次數
* 換回用戶：消費次數高於平均，但近期消費時間低於平均
* 潛在消費客戶：消費次數低於平均，但於近期曾有消費行為
* 重要價值客戶：消費次數高於平均，且曾於近日消費